# 策略思想

这篇文章想探索一下使用波动性来选择指数的可行性。基本思想是在A股的所有指数中，找到波动性最大的指数，然后在该指数上使用智能网格策略来验证投资效果

# 导入依赖包

In [8]:
import pickle
import akshare as ak
import talib
from scipy.signal import find_peaks
import pandas as pd
import numpy as np 
from datetime import datetime as dt, timedelta as td
from scipy.signal import argrelextrema 

# 导入数据

这里我预先缓存了akshare的数据，第一次使用的话先调用《缓存数据.ipynb》文件缓存一下数据

In [9]:
with open("cache-2024-09-01.pk", "rb") as fp:
    data = pickle.load(fp)

# 计算ATR指标

我们在这里使用ATR指标来衡量指数曲线的波动性。

为了忽略不同指数不同价格的影响，这里统一归一化之后计算，这样计算出来的数据才有可比性

以下是文心一言给出的ATR指标释义： 
>ATR指标全称为Average True Range，中文名称为平均真实波幅，它是一种技术指标，主要用于衡量股票、期货、外汇等交易品种的波动性。ATR指标最初由J. Welles Wilder Jr.于1978年在其所著的《新概念技术分析》(New Concepts in Technical Trading Systems)中提出。以下是对ATR指标的详细解释：
一、ATR指标的定义与计算
定义：ATR指标是取一定时间周期内的股价波动幅度的移动平均值，用于显示市场变化率的指标。
计算方式：首先计算当前交易日的TR（True Range）值，即当前交易日最高价与最低价之差、前一日收盘价与当日最高价之差、前一日收盘价与当日最低价之差中的最大值，然后通过对TR值进行平均计算得出ATR值。
二、ATR指标的应用
衡量市场波动性：ATR值通常被解释为市场波动性的度量，ATR值越大，表示市场波动越剧烈；ATR值越小，表示市场波动越平缓。
设置止损与止盈：ATR指标可用于设置止损点和止盈点。例如，在短线交易中，可以将止损点设置在入场点下方0.8ATR的位置；在长线交易中，则可以使用2～3ATR来设置止损点。同时，ATR也可以帮助交易者设定合理的止盈目标，避免过早离场或贪心不足。
分配仓位：ATR指标还可以用于指导仓位分配。在波动性较大的市场中，ATR值较大，此时应适当减少仓位以降低风险；反之，在波动性较小的市场中，ATR值较小，可以适当增加仓位以获取更多收益。
判断市场趋势：虽然ATR指标本身不具备判断市场趋势的能力，但它可以结合其他趋势指标如移动平均线一起使用。例如，当ATR值增大且伴随价格突破关键阻力位时，可能预示着市场趋势的转变。
识别市场反转：ATR指标在市场底部通常能达到一个较高的价值，这可能是由于惊恐购买所驱使的价格剧烈下跌所致。此外，当ATR值从下向上穿越移动平均线或从上向下穿越移动平均线时，可能预示着市场趋势的逆转。但具体如何转变还需结合其他趋势类指标进行综合研判。
三、ATR指标的局限性
滞后性：ATR指标是基于历史价格波动计算出来的，因此具有一定的滞后性，不能准确预测市场未来的波动。
需结合其他指标：ATR指标一般不单独使用，而是需要结合其他指标和技术分析方法来进行交易决策。例如，在判断市场趋势时，需要结合移动平均线等趋势类指标；在设置止损点时，也需要考虑市场的支撑位和阻力位等因素。
四、总结
ATR指标是一种重要的技术分析工具，它能够帮助交易者衡量市场波动性、设置止损与止盈、分配仓位以及判断市场趋势等。然而，在使用ATR指标时需要注意其滞后性和局限性，并结合其他指标和技术分析方法来进行综合研判。

调用工具库talib计算ATR指标，先把所有指数的ATR值计算出来

注意：talib这个库可能需要自己编译安装，官网地址：https://ta-lib.org/install/

In [32]:
atr_map = {}
for name, df in data.items():
    df_norm = (df - df.min()) / (df.max() - df.min())
    atr = talib.ATR(df_norm['High'], df_norm['Low'], df_norm['Close'], timeperiod=14)
    atr_map[name] = atr.mean()


# 筛选

按照ATR指标排序取最大的指数，这里筛选出来波动最小和最大如下

|类型    |标的代码|标的名称|波定性值|
|-------|-------|-------|------|
|波动最小|000101|5年信用|0.0007|
|波动最大|399986|中证值银行指数值|0.0635|

5年信用波动低这个很好理解，可是中证银行是波动最大的确实让人比较意外！

In [31]:
atr_meta = sorted(atr_map.items(), key=lambda x:x[1])
print(atr_meta[0], atr_meta[-1])

index = ak.index_stock_info()
print(index[index.index_code==atr_meta[0][0]])
print(index[index.index_code==atr_meta[-1][0]])

('000101', 0.0007133635875336833) ('399986', 0.0635468784624781)
   index_code display_name publish_date
87     000101         5年信用   2013-04-01
    index_code display_name publish_date
711     399986       中证银行指数   2013-07-15


## 定义一个函数，按照窗口大小过滤极小值点，仅保留窗口内的最小值  

In [33]:

def filter_minima_by_window(df, window_size):  
    peaks_min, _ = find_peaks(-df.Close, distance=window_size)
    peaks_max, _ = find_peaks(df.Close, distance=window_size)
    return df.iloc[peaks_min], df.iloc[peaks_max]

# 指标

接下来我们计算几个重点指标，包括：移动平均、atr（波动性）、adx（动量）、period（周期性）、布林带、极大极小值点

In [34]:
# 移动平均
ema = talib.EMA(df.Close)
ma5 = talib.MA(df.Close, 5)
ma20 = talib.MA(df.Close, 20)

# 波动性
atr = talib.ATR(df.High, df.Low, df.Close, timeperiod=14)

# 动量
adx = talib.ADX(df.High, df.Low, df.Close, timeperiod=14)

# 周期
period = talib.HT_TRENDMODE(df.Close)

xs = []
prev = period.iloc[0]
prev_dt = period.index[0]
ys = [prev]

for i, v in enumerate(period):
    if v != prev:
        xs.append([prev_dt,period.index[i]])
        ys.append(v)
        prev_dt = period.index[i]
        prev = v
[(x, y) for x, y in zip(xs, ys)]

# 布林带
upper, middle, lower = talib.BBANDS(df.Close, matype=talib.MA_Type.T3)

# 极小、极大值点
minima, maxmi = filter_minima_by_window(df, 30)

In [18]:
from pyecharts.charts import Line, Bar, Grid, Scatter
from pyecharts import options as opts
import talib
from pyecharts.globals import ThemeType
from pyecharts.globals import CurrentConfig, NotebookType

CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

# 折线图
line = (
    # 创建一个折线图对象
    Line()
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(name="Date"),
        yaxis_opts=opts.AxisOpts(name="收盘价",min_=df.Close.min()*0.9),
        title_opts=opts.TitleOpts(title=str(name)) 
    )
    # 添加数据
    .add_xaxis(df.index.map(lambda x: x.strftime("%Y-%m-%d")).tolist())
    .add_yaxis("Close", df.Close,label_opts=opts.LabelOpts(is_show=False))
    # .add_yaxis("MA5", ma5, label_opts=opts.LabelOpts(is_show=False))
    # .add_yaxis("MA20", data["MA30"], label_opts=opts.LabelOpts(is_show=False))
    # .add_yaxis("upper", upper, label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False)
    # .add_yaxis("lower", lower, label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False)
    # .add_yaxis("middle", middle, label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False)
)

scatter = (
    Scatter()
    .add_xaxis(minima.index.map(lambda x:x.strftime("%Y-%m-%d")).tolist())
    .add_yaxis("min", minima.Close.tolist(), symbol_size=8, label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
        yaxis_opts=opts.AxisOpts(type_="value"),
    )  
)
line.overlap(scatter)

scatter_m = (
    Scatter()
    .add_xaxis(maxmi.index.map(lambda x:x.strftime("%Y-%m-%d")).tolist())
    .add_yaxis("max", maxmi.Close.tolist(), symbol_size=8, label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
        yaxis_opts=opts.AxisOpts(type_="value"),
    )  
)
scatter_m.set_series_opts(itemstyle_opts=opts.ItemStyleOpts(color="red")) 
line.overlap(scatter_m)

# 获取当前日期  
now = dt.now()    
# 今年以来的数据  
start_of_year = dt(now.year, 1, 1)  
y1 = df[start_of_year:]  

three_years_ago_accurate = now - pd.DateOffset(years=3)  
y3 = df[three_years_ago_accurate:]  

five_years_ago_accurate = now - pd.DateOffset(years=5)  
y5 = df[five_years_ago_accurate:]  
  
ten_years_ago_accurate = now - pd.DateOffset(years=10)  
y10 = df[ten_years_ago_accurate:]

line1 = (
    # 创建一个折线图对象
    Line()
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(name="Date"),
        yaxis_opts=opts.AxisOpts(name="今年以来",min_=y1.Close.min()*0.9),
        title_opts=opts.TitleOpts(title=str(name)) 
    )
    # 添加数据
    .add_xaxis(y1.index.map(lambda x: x.strftime("%Y-%m-%d")).tolist())
    .add_yaxis("y1", y1.Close,label_opts=opts.LabelOpts(is_show=False))
    # .add_yaxis("MA5", data["MA5"], label_opts=opts.LabelOpts(is_show=False))
    # .add_yaxis("MA20", data["MA30"], label_opts=opts.LabelOpts(is_show=False))
    # .add_yaxis("upper", upper, label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False)
    # .add_yaxis("lower", lower, label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False)
    # .add_yaxis("middle", middle, label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False)
)
scatter1 = (
    Scatter()
    .add_xaxis(minima[start_of_year:].index.map(lambda x:x.strftime("%Y-%m-%d")).tolist())
    .add_yaxis("min", minima[start_of_year:].Close.tolist(), symbol_size=8, label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
        yaxis_opts=opts.AxisOpts(type_="value"),
    )  
)
line1.overlap(scatter1)

scatter_m1 = (
    Scatter()
    .add_xaxis(maxmi[start_of_year:].index.map(lambda x:x.strftime("%Y-%m-%d")).tolist())
    .add_yaxis("max", maxmi[start_of_year:].Close.tolist(), symbol_size=8, label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
        yaxis_opts=opts.AxisOpts(type_="value"),
    )  
)
scatter_m1.set_series_opts(itemstyle_opts=opts.ItemStyleOpts(color="red")) 
line1.overlap(scatter_m1)

line3 = (
    # 创建一个折线图对象
    Line()
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(name="Date"),
        yaxis_opts=opts.AxisOpts(name="过去3年",min_=y3.Close.min()*0.9),
        title_opts=opts.TitleOpts(title=str(name)) 
    )
    # 添加数据
    .add_xaxis(y3.index.map(lambda x: x.strftime("%Y-%m-%d")).tolist())
    .add_yaxis("Close", y3.Close,label_opts=opts.LabelOpts(is_show=False))
    # .add_yaxis("MA5", data["MA5"], label_opts=opts.LabelOpts(is_show=False))
    # .add_yaxis("MA20", data["MA30"], label_opts=opts.LabelOpts(is_show=False))
    # .add_yaxis("upper", upper, label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False)
    # .add_yaxis("lower", lower, label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False)
    # .add_yaxis("middle", middle, label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False)
)
scatter3 = (
    Scatter()
    .add_xaxis(minima[three_years_ago_accurate:].index.map(lambda x:x.strftime("%Y-%m-%d")).tolist())
    .add_yaxis("min", minima[three_years_ago_accurate:].Close.tolist(), symbol_size=8, label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
        yaxis_opts=opts.AxisOpts(type_="value"),
    )  
)
scatter_m3 = (
    Scatter()
    .add_xaxis(maxmi[three_years_ago_accurate:].index.map(lambda x:x.strftime("%Y-%m-%d")).tolist())
    .add_yaxis("max", maxmi[three_years_ago_accurate:].Close.tolist(), symbol_size=8, label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
        yaxis_opts=opts.AxisOpts(type_="value"),
    )  
)
scatter_m3.set_series_opts(itemstyle_opts=opts.ItemStyleOpts(color="red")) 
line3.overlap(scatter_m3)
line3.overlap(scatter3)

line5 = (
    # 创建一个折线图对象
    Line()
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(name="Date"),
        yaxis_opts=opts.AxisOpts(name="过去5年",min_=y5.Close.min()*0.9),
        title_opts=opts.TitleOpts(title=str(name)) 
    )
    # 添加数据
    .add_xaxis(y5.index.map(lambda x: x.strftime("%Y-%m-%d")).tolist())
    .add_yaxis("Close", y5.Close,label_opts=opts.LabelOpts(is_show=False))
    # .add_yaxis("MA5", data["MA5"], label_opts=opts.LabelOpts(is_show=False))
    # .add_yaxis("MA20", data["MA30"], label_opts=opts.LabelOpts(is_show=False))
    # .add_yaxis("upper", upper, label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False)
    # .add_yaxis("lower", lower, label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False)
    # .add_yaxis("middle", middle, label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False)
)

line10 = (
    # 创建一个折线图对象
    Line()
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(name="Date"),
        yaxis_opts=opts.AxisOpts(name="过去10年",min_=y10.Close.min()*0.9),
        title_opts=opts.TitleOpts(title=str(name)) 
    )
    # 添加数据
    .add_xaxis(y10.index.map(lambda x: x.strftime("%Y-%m-%d")).tolist())
    .add_yaxis("Close", y10.Close,label_opts=opts.LabelOpts(is_show=False))
    # .add_yaxis("MA5", data["MA5"], label_opts=opts.LabelOpts(is_show=False))
    # .add_yaxis("MA20", data["MA30"], label_opts=opts.LabelOpts(is_show=False))
    # .add_yaxis("upper", upper, label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False)
    # .add_yaxis("lower", lower, label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False)
    # .add_yaxis("middle", middle, label_opts=opts.LabelOpts(is_show=False), is_symbol_show=False)
)


# ATR
bar = (
    Bar()
    .add_xaxis(df.index.map(lambda x: x.strftime("%Y-%m-%d")).tolist())
    .add_yaxis("ATR", atr.tolist(),label_opts=opts.LabelOpts(is_show=False))
    .add_yaxis("ADX", adx.tolist(), label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(title_opts=opts.TitleOpts(title="ATR",pos_top="82%"),
                     legend_opts=opts.LegendOpts(pos_top="82%"))
)


grid = (
    Grid(init_opts=opts.InitOpts(width="2400px",height="1200px"))
    .add(line, grid_opts=opts.GridOpts(pos_bottom="70%"))
    .add(line1, grid_opts=opts.GridOpts(pos_bottom="40%",pos_top="34%",pos_right="52%"))
    .add(line3, grid_opts=opts.GridOpts(pos_bottom="40%",pos_top="34%",pos_left="52%"))
    .add(line5, grid_opts=opts.GridOpts(pos_bottom="20%",pos_top="64%",pos_right="52%"))
    .add(line10, grid_opts=opts.GridOpts(pos_bottom="20%",pos_top="64%",pos_left="52%"))

    # 通过设置图形相对位置，来调整是整个并行图是竖直放置，还是水平放置
    .add(bar, grid_opts=opts.GridOpts(pos_top="84%"))
)
grid.load_javascript()

In [19]:
grid.render_notebook()